# 02-02 Dimensionen Aufgabe

## Hinweise zur Übung

In der Demo wurde gezeigt, wie eine einfache (flache) und eine etwas komplexere (hiearchische) Dimensionstabelle aus Quelldaten im Stagingbereich erzeugt und in die Reporting-Datenbank geladen werden können.

In dieser Übung übertragen Sie die Vorgehensweise auf je eine einfache und eine komplexere Dimension mit anderen Quelldaten.

Die Tabellenstruktur der zu erstellenden Dimensionstabellen inkl. der Spalten (nicht jedoch Datentypen) geht aus dem ER-Diagramm im `data`-Verzeichnis hervor. Die Datentypen wiederum können in den Dokumentationen der jeweiligen Qeulldatenbereiche (Bielefeld, Mannheim, Dimensionen) nachgeschlagen werden.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql

In [ ]:
import os
import sys
import urllib.request
%load_ext sql

In [ ]:
# Konfiguration
base_url_quellen = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/07-ETL+DWH/data"
base_url_staging = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_bielefeld.sqlite",  "stg_bielefeld.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_mannheim.sqlite",   "stg_mannheim.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_dimensions.sqlite", "stg_dimensions.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/staging.sqlite",        "staging.sqlite")


In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_stg_bielefeld  = base_url_staging + "stg_bielefeld.sqlite"
db_path_stg_mannheim   = base_url_staging + "stg_mannheim.sqlite"
db_path_stg_dimensions = base_url_staging + "stg_dimensions.sqlite"
db_path_staging        = base_url_staging + "staging.sqlite"
db_path_reporting      = base_url_staging + "reporting.sqlite"

db_url_stg_bielefeld  = "sqlite:///" + db_path_stg_bielefeld
db_url_stg_mannheim   = "sqlite:///" + db_path_stg_mannheim
db_url_stg_dimensions = "sqlite:///" + db_path_stg_dimensions
db_url_staging        = "sqlite:///" + db_path_staging
db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_stg_bielefeld
%sql $db_url_stg_mannheim
%sql $db_url_stg_dimensions
%sql $db_url_staging
%sql $db_url_reporting

In [ ]:
%%sql $db_url_staging
ATTACH DATABASE :db_path_stg_bielefeld  AS bielefeld;
ATTACH DATABASE :db_path_stg_mannheim   AS mannheim;
ATTACH DATABASE :db_path_stg_dimensions AS dimensions;
ATTACH DATABASE :db_path_reporting      AS reporting;

## Aufgabe: Dimensionstabelle D_PROZEDUR generieren

Generieren Sie die Dimensionstabelle D_PROZEDUR gemäß den Vorgaben des DWH-Datenmodells! Nutzen sie dazu die Quelldaten aus den Prozedurentabellen des Bielefelder & Mannheimer Quelldatensatzes und führen sie zusammen. Prüfen Sie anschließend, ob die Tabelle korrekt befüllt wurde.

### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Reporting-Datenbank (leer) an.

### 2. Daten aus den Quelltabellen auslesen & in Zieltabelle schreiben

In diesem Schritt lesen wir mit `SELECT`-Statements die Daten aus den Quelltabellen beider Standorte aus.

Wir benennen bei der Abfrage die Spalten mit der `AS`-Klausel nach den Vorgeben unseres Datenmodells um.

Um eindeutige Einträge in der Zieltabelle zu erhalten, nutzen wir die `DISTINCT`-Klausel.

### 3. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt erstellt wurde.


## Aufgabe: Dimensionstabelle D_DIAGNOSE generieren

Generieren Sie die Dimensionstabelle D_DIAGNOSE mit ihren 3 Ebenen gemäß den Vorgaben des DWH-Datenmodells! Nutzen Sie dazu die Quelldaten der Tabelle "CONDITIONS_ICD10CM" im Dimensionsbereich. Prüfen Sie anschließend, ob die Tabelle korrekt befüllt wurde.

### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Reporting-Datenbank (leer) an.

### 2. Daten aus der Quelltabelle auslesen & in Zieltabelle schreiben

Die Dimension D_DIAGNOSE hat laut DWH-Datenmodell die 3 folgenden Ebenen:
* KAPITEL: Kapitel des ICD-Katalogs
* ICD: Einzelne ICD-Codes (Blatt-Ebene von ICD)
* SNOMED: Zugeordnete SNOMED-Codes

Die LOINC-Klasse ist in den Quelldaten (wie oben bei der Fallart) nur mit einem textuellen Code, jedoch keiner ID abgebildet. Da der Code sprechend ist, tragen wir den Text der Klasse identisch in die _ID- und _NAME-Spalten ein.

Da die Quelldaten für beide Ebenen der Hierarchie aus der gleichen Quelltabelle stammen, besteht die Aufgabe hier nur darin, die Spalten gemäß den Vorgaben des DWH-Datenmodells umzubenennen und in die Zieltabelle per `INSERT INTO  ... SELECT ...` auszuliefern.

### 3. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt erstellt wurde.